In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

# Define the problem instance
num_items = 42
bin_capacity = 1000
mutation_rate = 0.001
crossover_rate = 0.6
generations = 10

item_weights = {
    200: 3, 199: 5, 198: 4, 197: 1, 195: 1, 193: 4, 192: 1, 188: 1, 187: 1, 186: 3, 
    185: 3, 184: 2, 183: 2, 182: 1, 181: 1, 180: 3, 179: 2, 178: 6, 177: 2, 176: 4, 
    175: 1, 173: 4, 172: 4, 170: 1, 169: 3, 168: 4, 167: 1, 165: 3, 164: 1, 163: 2, 
    162: 4, 161: 1, 160: 3, 159: 3, 158: 1, 157: 3, 155: 2, 154: 3, 153: 1, 152: 3, 
    151: 1, 150: 1
}

print(item_weights)


def binary_to_decimal(binary_list):
    decimal_list = []
    for binary_nums in binary_list:
        decimal_nums = []
        for binary_num in binary_nums:
            if isinstance(binary_num, list):
                binary_num = ''.join([str(bit) for bit in binary_num])
            decimal_num = int(str(binary_num), 2)
            decimal_nums.append(decimal_num)
        decimal_list.append(decimal_nums)
    return decimal_list

def decimal_to_binary(decimal_list):
    bits = ''
    bits_list = []
    for sublist in decimal_list:
        sublist_bits = []
        for num in sublist:
            sublist_bits.append(format(num, '08b'))
        bits_list.append(sublist_bits)

    binary_list = '[' + ', '.join(['[' + ', '.join(sublist) + ']' for sublist in bits_list]) + ']'
    return binary_list
    
def sum_subsets(subsets):
    sums = []
    for subset in subsets:
        subset_sum = sum(subset)
        if subset_sum > bin_capacity:
            subset_sum = 0
        sums.append(subset_sum)
    return sums


def mutate_individual(individual, mutation_rate):
    mutated_individual = individual.copy()
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            # Flip the bit at position i
            mutated_individual[i] = 1 - mutated_individual[i]
    return mutated_individual

def mutate_population(population, mutation_rate):
    mutated_population = []
    for individual in population:
        mutated_individual = mutate_individual(individual, mutation_rate)
        mutated_population.append(mutated_individual)
    return mutated_population

# Function for crossover
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, 7)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    else:
        return parent1, parent2

{200: 3, 199: 5, 198: 4, 197: 1, 195: 1, 193: 4, 192: 1, 188: 1, 187: 1, 186: 3, 185: 3, 184: 2, 183: 2, 182: 1, 181: 1, 180: 3, 179: 2, 178: 6, 177: 2, 176: 4, 175: 1, 173: 4, 172: 4, 170: 1, 169: 3, 168: 4, 167: 1, 165: 3, 164: 1, 163: 2, 162: 4, 161: 1, 160: 3, 159: 3, 158: 1, 157: 3, 155: 2, 154: 3, 153: 1, 152: 3, 151: 1, 150: 1}


In [9]:
def pick_random_integer():
    choices = list(item_weights.keys())
    weights = list(item_weights.values())
    random_integer = random.choices(choices, weights=weights)[0]
    return random_integer

def pick_integers():
    chosen_integers = []
    sublists = [[] for _ in range(num_items)]  # Create 42 empty sublists
    for _ in range(len(item_weights)):
        random_integer = pick_random_integer()
        chosen_integers.append(random_integer)
        sublist_index = len(chosen_integers) % num_items  # Determine the index of the sublist
        sublists[sublist_index].append(random_integer)  # Append the integer to the corresponding sublist
        item_weights[random_integer] -= 1  # Decrease the count of the chosen integer
        if item_weights[random_integer] == 0:
            del item_weights[random_integer]  # Remove the integer if its count becomes 0
            
    # Append remaining integers to sublists in sequence
    remaining_integers = list(item_weights.keys())
    for i, integer in enumerate(remaining_integers):
        sublist_index = i % 42
        sublists[sublist_index].append(integer)
        
    return sublists

In [10]:
chosen_sublists = pick_integers()
print("Chosen sublists:", chosen_sublists)

# Fuction to calculate fitness
def calculate_fitness(population):
        indiv_fitness = sum_subsets(population)
        return indiv_fitness 

Chosen sublists: [[168, 200], [160, 199], [186, 198], [200, 197], [162, 193], [165, 192], [178, 188], [157, 186], [167, 185], [152, 184], [154, 181], [159, 180], [185, 179], [193, 178], [173, 177], [162, 176], [195, 175], [200, 173], [198, 172], [168, 169], [159, 168], [176, 165], [183, 163], [153, 162], [178, 161], [187, 160], [198, 159], [157, 158], [177, 157], [164, 155], [180, 154], [176, 152], [176, 150], [170], [199], [168], [180], [182], [178], [151], [154], [183]]


In [11]:
# Main genetic algorithm loop
for generation in range(generations):
    fitness_values = calculate_fitness(chosen_sublists)
    fitness_bin_history = len(chosen_sublists)
    print("Bins used: " + str(fitness_bin_history))
    #fitness_bin_history.append(fitness_values)

   # Convert fitness values to probabilities
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]

    # Select parents based on fitness
    parents_indices = np.random.choice(len(chosen_sublists), size=len(chosen_sublists) // 2, p=probabilities)
    parents = [chosen_sublists[i] for i in parents_indices]

    # Create next generation through crossover and mutation
    children = []
    for i in range(0, 5):
        child1, child2 = crossover(parents[i], parents[i + 1])
        child1 = mutate_individual(child1, mutation_rate)
        child2 = mutate_individual(child2, mutation_rate)
        children.extend([child1, child2])

    chosen_sublists[:len(chosen_sublists) // 2] = parents
    chosen_sublists[len(chosen_sublists) // 2:] = children

Bins used: 42
Bins used: 31
Bins used: 25
Bins used: 22
Bins used: 21
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
Bins used: 20
